# Recording

In [5]:
import pyaudio
import wave
 
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 96000
CHUNK = 1024
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "file.wav"

dev_index = 6

# find SDR-Widget
def find_device():
    p = pyaudio.PyAudio()
    info = p.get_host_api_info_by_index(0)
    numdevices = info.get('deviceCount')
    devnumber=-1
    for i in range(0, numdevices):
        if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))
            if (p.get_device_info_by_host_api_device_index(0, i).get('name').find('Yoyodyne')==0):
                devnumber=i
    p.terminate()
    return (devnumber)

#dev_index = find_device()
dev_index = 6
print('SDR Widget:', dev_index)

audio = pyaudio.PyAudio()
 
# start Recording
stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,
                frames_per_buffer=CHUNK,
                input_device_index = dev_index)
print ("recording...")
frames = []
 
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)
print ("finished recording")
 
 
# stop Recording
stream.stop_stream()
stream.close()
audio.terminate()
 
waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
waveFile.setnchannels(CHANNELS)
waveFile.setsampwidth(audio.get_sample_size(FORMAT))
waveFile.setframerate(RATE)
waveFile.writeframes(b''.join(frames))
waveFile.close()

SDR Widget: 6
recording...
finished recording


## List of sound devices

In [4]:
def find_device():
    p = pyaudio.PyAudio()
    info = p.get_host_api_info_by_index(0)
    numdevices = info.get('deviceCount')
    devnumber=-1
    for i in range(0, numdevices):
        if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))
            if (p.get_device_info_by_host_api_device_index(0, i).get('name').find('Yoyodyne')==0):
                devnumber=i
    p.terminate()
    return (devnumber)

print('SDR Widget:',find_device())

Input Device id  0  -  HDA Intel PCH: ALC3246 Analog (hw:0,0)
Input Device id  6  -  Yoyodyne SDR-Widget: USB Audio (hw:1,0)
Input Device id  7  -  WD15 Dock: USB Audio (hw:2,0)
Input Device id  9  -  sysdefault
Input Device id  15  -  samplerate
Input Device id  16  -  speexrate
Input Device id  17  -  pulse
Input Device id  18  -  upmix
Input Device id  19  -  vdownmix
Input Device id  21  -  default
SDR Widget: 6


# Noise detect

In [ ]:
import pyaudio
import math
import struct
import wave
import time
import os

Threshold = 200

SHORT_NORMALIZE = (1.0/32768.0)
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 96000
swidth = 2

TIMEOUT_LENGTH = 5 # in seconds

f_name_directory = r'.'

class Recorder:

    @staticmethod
    def rms(frame):
        count = len(frame) / swidth
        format = "%dh" % (count)
        shorts = struct.unpack(format, frame)

        sum_squares = 0.0
        for sample in shorts:
            n = sample * SHORT_NORMALIZE
            sum_squares += n * n
        rms = math.pow(sum_squares / count, 0.5)

        return rms * 1000

    def __init__(self):
        self.dev_index = -1
        self.p = pyaudio.PyAudio()
        self.info = self.p.get_host_api_info_by_index(0)
        self.numdevices = self.info.get('deviceCount')
        for i in range(0, self.numdevices):
            if (self.p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
                print("Input Device id ", i, " - ", self.p.get_device_info_by_host_api_device_index(0, i).get('name'))
                if (self.p.get_device_info_by_host_api_device_index(0, i).get('name').find('Yoyodyne')==0):
                    self.dev_index = i
        print('SDR Widget:', self.dev_index)
        self.stream = self.p.open(format=FORMAT,
                                  channels=CHANNELS,
                                  rate=RATE,
                                  input=True,
                                  frames_per_buffer=CHUNK,
                                  input_device_index = self.dev_index)

    def record(self):
        print('Noise detected, recording beginning')
        rec = []
        current = time.time()
        end = time.time() + TIMEOUT_LENGTH

        while current <= end:

            data = self.stream.read(CHUNK)
            if self.rms(data) >= Threshold: end = time.time() + TIMEOUT_LENGTH

            current = time.time()
            rec.append(data)
        self.write(b''.join(rec))

    def write(self, recording):
        #n_files = len(os.listdir(f_name_directory))
        n_files = str(time.time()).split('.')[0]
        
        filename = os.path.join(f_name_directory, '{}.wav'.format(n_files))

        wf = wave.open(filename, 'wb')
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(self.p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(recording)
        wf.close()
        print('Written to file: {}'.format(filename))
        print('Returning to listening')



    def listen(self):
        print('Listening beginning')
        while True:
            input = self.stream.read(CHUNK)
            rms_val = self.rms(input)
            if rms_val > Threshold:
                self.record()

                
    def close(self):
        self.stream.close()
                
while (True):
    try:
        a = Recorder()

        a.listen()
    except:
        a.close()
        pass


Input Device id  0  -  HDA Intel PCH: ALC3246 Analog (hw:0,0)
Input Device id  6  -  Yoyodyne SDR-Widget: USB Audio (hw:1,0)
Input Device id  7  -  WD15 Dock: USB Audio (hw:2,0)
Input Device id  9  -  sysdefault
Input Device id  15  -  samplerate
Input Device id  16  -  speexrate
Input Device id  17  -  pulse
Input Device id  18  -  upmix
Input Device id  19  -  vdownmix
Input Device id  21  -  default
SDR Widget: 6
Listening beginning
Noise detected, recording beginning
Written to file: ./1608580668.wav
Returning to listening


In [9]:
import socket
import pyaudio

# Socket
HOST = socket.gethostname()
PORT = 3701

# Audio
p = pyaudio.PyAudio()
CHUNK = 1024 * 4
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 96000
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                output=True,
                frames_per_buffer=CHUNK)

with socket.socket() as server_socket:
    server_socket.connect((HOST, PORT))
    #server_socket.listen(1)
    #conn, address = server_socket.accept()
    #print("Connection from " + address[0] + ":" + str(address[1]))

    data = server_socket.recv(4096)
    for n in range(4):
        try:
            data = server_socket.recv(4096)
            stream.write(data)
        except socket.error:
            print("Client Disconnected")
            break

stream.stop_stream()
stream.close()
p.terminate()


In [11]:
import socket

HOST = 'localhost'  # The server's hostname or IP address
PORT = 3701        # The port used by the server

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    s.sendall(b'Hello, world')
    data = s.recv(1024)

#print('Received', data)
data

b'\x02\x06g;\x01\x00\x9c\xb7\x02\x96d;\x01\x00\n7\x028j;\x01@\x8c8\x02\xf6k;\x02@\xc98\x02\x9aj;\x01@\xbc8\x02\x82g;\x02\x00w8\x02\xf4g;\x02\x80X8\x02\xa0k;\x01\xc0\xbc8\x02tj;\x02\x00\xe07\x02Dk;\x02\x80\\8\x02\xb0h;\x01\x0058\x02Tg;\x02\x00\xf17\x02\x94d;\x02\x00\xdb7\x02\xaad;\x02\x00\xe0\xb6\x02\xf0c;\x01\x00\xa27\x02\xf8b;\x01\x00\x877\x02hc;\x01\x00\xb67\x02\x96c;\x02\x00t\xb8\x02\xfcb;\x01\x00\x87\xb7\x02:b;\x02\x00\xdb7\x02ta;\x01\x8038\x02\x84];\x02\x00\xce\xb7\x02Z\\;\x01\x00\x8e7\x02\xc8^;\x01\x00/8\x02\x18a;\x02\x00O8\x02*^;\x01\x00\xb57\x02D_;\x01\x00\x0c7\x02B`;\x02\x00P\xb6\x02\xd6\\;\x02\x00\xd37\x02\xea];\x02\x00r\xb7\x02\xf4[;\x01\x00\xb8\xb6\x02\xf6^;\x02\x00\xc17\x02\x16Y;\x01\x00.8\x02jY;\x01\x80:8\x02V[;\x02\x00M\xb8\x02\x92X;\x01@\xa8\xb8\x02rZ;\x01\x80\x93\xb8\x02$Z;\x02\x80a8\x02\x04^;\x01\x80\x0b\xb8\x02^];\x02\x00\xd67\x02^];\x02\x80\xef8\x02\x0e];\x02\x80D8\x02([;\x02\x80J\xb8\x02\x18\\;\x01\x80\x0f\xb8\x02\xaa];\x01\x00\x08\xb6\x02\xd4`;\x01\x00\x94\xb7\x02